# Coursera Spacecraft Dynamics and Control - Capstone Mars Misson
## Module 5

In [1]:
import numpy as np
import sys
from plotly import graph_objects as pgo
sys.path.append("../")
sys.path.append("../..")

import Sat_Orbits as SO
import attitude_math as am

In [2]:
get_state_at_time = lambda df, t: df.loc[df['t'] == t].to_numpy()

## Programming Assignment: Task 8: Sun Pointing Control
First, the individual control pointing modes are developed and tested on their own. Next, they are combined into a full mission scenario simulation in section 4.11. In the current task, use the initial spacecraft attitude and orbit conditions in section 3.1 and Table 2 and assume the spacecraft is to engage directly into a sun-pointing mode starting at $t_0$. The attitude control law is the simple PD control shown in Eq. (4).

Your tasks are:
1. Numerically implement this control law in your earlier simulation and ensure that sun-pointing is achieved with the desired closed-loop performance.
2. Use linearized closed-loop dynamics of a regular problem, such as this sun pointing control, to determine the $K$ and $P$ feedback gains such that the slowest decay response time (i.e., time for tracking errors to be $1/e$ the original size) is 120 seconds. This means all decay time constants should be 120 seconds or less. Further, the closed-loop response for all BN components should be either critically damped or under-damped. Thus, at least one mode must be critically damped with $\zeta=1$, while the other modes will have $\zeta<1$.
3. Validate the response by providing the BN states at $t = 15s$, $t = 100s$, $t = 200s$, and $t = 400s$. Note that you must always provide the MRP corresponding to the short rotation.


Your tasks are:

- Numerically implement this control law in your earlier simulation and ensure that sun-pointing is achieved with the desired closed-loop performance.

- Use linearized closed-loop dynamics of a regular problem, such as this sun pointing control, to determine the feedback gains K and P such that the slowest decay response time (i.e., time for tracking errors to be 1/e the original size) is 120 seconds. This means all decay time constants should be 120 seconds or less. Further, the closed-loop response for all σB/N components should be either critically damped or underdamped. Thus, at least one must be critically damped with ξ = 1, while the other modes will have ξ ≤ 1.

- Validate the response by providing the σB/N states at t = 15s, 100s, 200s, and 400s. Note that you must always provide the MRP corresponding to the short rotation.

Submit the following for grading on Coursera:

### Part 1: 
Submit a plain text file listing first the P feedback gain and then the K feedback gain. Tolerance: ±0.0001.



In [3]:
Iis = SO.LMO_sat.MOI.diagonal()
Ps = 2*Iis/120
print(f"P Candidates: {Ps}")
P = max(Ps)
E_target = 1
K_s = (P/E_target)**2/Iis
print(f"K Candidates: {K_s}")
K = max(K_s)

print(f"Selected Gains\tK = {K},\tP = {P}")
Ts = 2*Iis/P
Eis = P/np.sqrt(K*Iis)
print(f"Time decay constants: {Ts}\nDamping ratio: {Eis}")

with open("SCDC-M3-T8-P1.txt","w") as f:
    f.write(" ".join([str(P), str(K)]))

def linear_control(X, Xr, t):
    if callable(Xr):
        Xr = Xr(t)
    sigma_BR = am.DCM_2_MRP(am.MRP_2_DCM(X[0])@am.MRP_2_DCM(Xr[0]).transpose())
    omega_BR_B = X[1]-Xr[1]
    return - K*sigma_BR - P*omega_BR_B

P Candidates: [0.16666667 0.08333333 0.125     ]
K Candidates: [0.00277778 0.00555556 0.0037037 ]
Selected Gains	K = 0.005555555555555555,	P = 0.16666666666666666
Time decay constants: [120.  60.  90.]
Damping ratio: [0.70710678 1.         0.81649658]


### Part 2-5: Simulation 

In [4]:
SO.LMO_sat.reset_X()
# omega_0 = SO.LMO_sat.X()[1].transpose()
# H=SO.LMO_sat.MOI@omega_0
# print(f"H(0 s) = {H}")

t_step=1
target = np.vstack([SO.MRP_RsN, np.zeros(3)])#SO.omega_RsN])
data = SO.orbital_att_RK4int(SO.LMO_sat,
                      lambda x,t: SO.linear_control(x, 
                                                 target,
                                                 t, K, P),
                      np.zeros(3),
                      SO.LMO_sat.X(),
                      np.arange(0,400+t_step,t_step))
print(data)

fig = pgo.Figure()
sigma = target[0]
fig.add_scatter3d(x=data["sigma1"],
                  y=data["sigma2"],
                  z=data["sigma3"],
                  mode="lines")

lastval = data.iloc[-1].to_numpy()
print(lastval)
sigma_BR = am.DCM_2_MRP(am.MRP_2_DCM(lastval[1:4])@am.MRP_2_DCM(target[0]).transpose())
print(sigma_BR)
fig.add_scatter3d(x=[sigma[0]],
                  y=[sigma[1]],
                  z=[sigma[2]],
                  mode="markers")
fig.show()

         t    sigma1    sigma2    sigma3    omega1    omega2    omega3
0      0.0  0.300000 -0.400000  0.500000  0.017453  0.030543 -0.038397
1      1.0  0.298235 -0.381090  0.496846  0.017881  0.030386 -0.037217
2      2.0  0.296351 -0.362695  0.493923  0.018280  0.030237 -0.036049
3      3.0  0.294361 -0.344785  0.491223  0.018651  0.030095 -0.034895
4      4.0  0.292276 -0.327334  0.488737  0.018995  0.029957 -0.033754
..     ...       ...       ...       ...       ...       ...       ...
396  396.0 -0.010877 -0.719551 -0.684502 -0.000832  0.000076 -0.000485
397  397.0 -0.010682 -0.719372 -0.684899 -0.000825  0.000075 -0.000479
398  398.0 -0.010489 -0.719195 -0.685293 -0.000818  0.000074 -0.000473
399  399.0 -0.010299 -0.719018 -0.685682 -0.000811  0.000073 -0.000467
400  400.0 -0.010111 -0.718841 -0.686069 -0.000804  0.000072 -0.000461

[401 rows x 7 columns]
[ 4.00000000e+02 -1.01114031e-02 -7.18841470e-01 -6.86068657e-01
 -8.03911498e-04  7.17871175e-05 -4.61049148e-04]
[ 0.01166

### Part 2:
Submit a plain text file containing the MRP $\sigma_{B/N}$ at t = 15s. 

Tolerance: ±0.001.

In [5]:
sigma_BN = get_state_at_time(data, t=15)[0,1:4]
print(sigma_BN)
with open("SCDC-M3-T8-P2.txt","w") as f:
    f.write(" ".join(str(sigma) for sigma in sigma_BN.flatten()))

[ 0.26559696 -0.15982902  0.47332779]


### Part 3:
Submit a plain text file containing the MRP $\sigma_{B/N}$ at t = 100s. 

Tolerance: ±0.001.

In [6]:
sigma_BN = get_state_at_time(data, t=100)[0,1:4]
print(sigma_BN)
with open("SCDC-M3-T8-P3.txt","w") as f:
    f.write(" ".join(str(sigma) for sigma in sigma_BN.flatten()))

[0.16882716 0.54822575 0.57886626]


### Part 4:
Submit a plain text file containing the MRP $\sigma_{B/N}$ at t = 200s. 

Tolerance: ±0.001.

In [7]:
sigma_BN = get_state_at_time(data, t=200)[0,1:4]
print(sigma_BN)
with open("SCDC-M3-T8-P4.txt","w") as f:
    f.write(" ".join(str(sigma) for sigma in sigma_BN.flatten()))

[-0.11812732 -0.75785989 -0.59149202]


### Part 5:
Submit a plain text file containing the MRP $\sigma_{B/N}$ at t = 400s. 

Tolerance: ±0.001.

In [8]:
sigma_BN = get_state_at_time(data, t=400)[0,1:4]
print(sigma_BN)
with open("SCDC-M3-T8-P5.txt","w") as f:
    f.write(" ".join(str(sigma) for sigma in sigma_BN.flatten()))

[-0.0101114  -0.71884147 -0.68606866]


## Programming Assignment: Task 9: Nadir Pointing Control (10 points)
Next, the nadir pointing attitude mode is created and tested. Use the same gains K and P as developed for sun-pointing, and the PD control in Eq. (4). Your tasks are:

- Numerically implement the nadir pointing control mode and assume that at t0 nadir pointing is required, even though the satellite is in the sunlight at this time.

- Validate the response by providing the BN states at t = 15s, 100s, 200s, and 400s. Note that you must always provide the MRP corresponding to the short rotation.


In [12]:
SO.LMO_sat.reset_X()
# omega_0 = SO.LMO_sat.X()[1].transpose()
# H=SO.LMO_sat.MOI@omega_0
# print(f"H(0 s) = {H}")

t_step=1
#Xr = lambda t: np.vstack([am.DCM_2_MRP(SO.LMO_sat.R_RnN(t)),
#                          SO.LMO_sat.R_BN()@SO.LMO_sat.omega_RnN_N(t)])
def Xr(t):
    Xr = np.vstack([am.DCM_2_MRP(SO.LMO_sat.R_RnN(t)),
                          SO.LMO_sat.R_BN()@SO.LMO_sat.omega_RnN_N(t)])
    print(Xr[1])
    return Xr

control = lambda x, t: SO.linear_control(x, 
                                         Xr,
                                         t, K, P)
data = SO.orbital_att_RK4int(SO.LMO_sat,
                      control,
                      np.zeros(3),
                      SO.LMO_sat.X(),
                      np.arange(0,400+t_step,t_step))
print(data)

fig = pgo.Figure()
sigma = np.array([Xr(t).flatten() for t in np.arange(0,400+t_step,t_step)])

fig.add_scatter3d(x=sigma[:,0],
                  y=sigma[:,1],
                  z=sigma[:,2],
                  mode="markers",
                  marker={"size": 1})
fig.add_scatter3d(x=data["sigma1"],
                  y=data["sigma2"],
                  z=data["sigma3"],
                  marker={"size": 1})
fig.show()

[ 0.00060446 -0.00038553  0.00051852]
[ 0.00060257 -0.00035348  0.00054298]
[ 0.0005991  -0.0003219   0.00056597]
[ 0.00059425 -0.00029086  0.00058748]
[ 0.00058821 -0.00026041  0.00060751]
[ 0.00058115 -0.00023059  0.00062607]
[ 0.00057324 -0.00020143  0.00064319]
[ 0.00056463 -0.00017297  0.00065889]
[ 0.00055546 -0.00014521  0.00067323]
[ 0.00054587 -0.00011818  0.00068624]
[ 5.35965241e-04 -9.18853915e-05  6.97971046e-04]
[ 5.25863118e-04 -6.63212344e-05  7.08473514e-04]
[ 5.15659606e-04 -4.14902096e-05  7.17801490e-04]
[ 5.05443113e-04 -1.73876882e-05  7.26009761e-04]
[4.95292776e-04 5.99392771e-06 7.33153725e-04]
[4.85278999e-04 2.86647382e-05 7.39288760e-04]
[4.75464003e-04 5.06369286e-05 7.44469697e-04]
[4.65902375e-04 7.19243972e-05 7.48750362e-04]
[4.56641618e-04 9.25424239e-05 7.52183208e-04]
[0.00044772 0.00011251 0.00075482]
[0.00043918 0.00013184 0.00075671]
[0.00043104 0.00015055 0.00075789]
[0.00042334 0.00016866 0.00075842]
[0.00041608 0.00018619 0.00075834]
[0.0004092

**Your tasks are:**

Numerically implement the nadir pointing control mode and for now just assume that at 
$t_0$
nadir pointing is required, even though the satellite is in the sunlight at this time.

Validate the response by providing the 
$\sigma_{B/N}$
states at 
$t = 15s$, $t = 100s$, $t = 200s$, and $t = 400s$. Note that you must always provide the MRP corresponding to the short rotation.

Submit the following for grading in Coursera:

### Part 1:
Compute the 
$\sigma_{B/N}$
state at 
$t = 15s$

Tolerance: ±0.001

In [ ]:
t_measure = 15
sigma_BN = get_state_at_time(data, t=t_measure)[0,1:4]
print(F"sigma_BN(t={t_measure}) = {sigma_BN}")
with open("SCDC-M3-T9-P1.txt","w") as f:
    f.write(" ".join(str(sigma) for sigma in sigma_BN.flatten()))

sigma_BN(t=15) = [ 0.29092842 -0.19062173  0.45249325]


### Part 2:
Compute the 
$\sigma_{B/N}$
state at 
$t = 100s$

Tolerance: ±0.001

In [ ]:
t_measure = 100
sigma_BN = get_state_at_time(data, t=t_measure)[0,1:4]
print(F"sigma_BN(t={t_measure}) = {sigma_BN}")
with open("SCDC-M3-T9-P2.txt","w") as f:
    f.write(" ".join(str(sigma) for sigma in sigma_BN.flatten()))

sigma_BN(t=100) = [ 0.55459398 -0.12851198  0.12595436]


### Part 3:
Compute the 
$\sigma_{B/N}$
state at 
$t = 200s$

Tolerance: ±0.001

In [ ]:
t_measure = 200
sigma_BN = get_state_at_time(data, t=t_measure)[0,1:4]
print(F"sigma_BN(t={t_measure}) = {sigma_BN}")
with open("SCDC-M3-T9-P3.txt","w") as f:
    f.write(" ".join(str(sigma) for sigma in sigma_BN.flatten()))

sigma_BN(t=200) = [ 0.7698941  -0.43347643 -0.17245087]


### Part 4:
Compute the 
$\sigma_{B/N}$
state at 
$t = 400s$

Tolerance: ±0.001

In [ ]:
t_measure = 400
sigma_BN = get_state_at_time(data, t=t_measure)[0,1:4]
print(F"sigma_BN(t={t_measure}) = {sigma_BN}")
with open("SCDC-M3-T9-P4.txt","w") as f:
    f.write(" ".join(str(sigma) for sigma in sigma_BN.flatten()))

sigma_BN(t=400) = [-0.65628754  0.53259266  0.18560556]
